In [4]:
from transformers import DebertaV2Tokenizer, AutoTokenizer, DebertaV2ForMaskedLM
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset, load_from_disk
from tqdm.auto import tqdm

c:\Users\Matheus\Documents\Git\Longformer-PT\venv_lg_former\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
dataset = load_from_disk("c4ai-wik")

In [6]:
tokenizer = AutoTokenizer.from_pretrained("deberta_pt_tokenizer")

In [7]:
model = DebertaV2ForMaskedLM.from_pretrained("microsoft/deberta-v3-base", vocab_size=tokenizer.vocab_size, ignore_mismatched_sizes=True)

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForMaskedLM: ['mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'deberta.embeddings.position_embeddings.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

### Splitting data by max_lenght

In [10]:
MAX_POS_EMBD = model.config.max_position_embeddings
WINDOW = 200

In [15]:
from functools import partial

def _tokenizer(sample, tokenizer, window, max_lenght):

    tkn_text = tokenizer(sample['text'], add_special_tokens=False)

    dev_train_dataset = []
    for i in range(0, len(tkn_text['input_ids']), window):
        dev_train_dataset.append(tkn_text['input_ids'][i:max_lenght+i])

    return {k: v for k, v in enumerate(dev_train_dataset)}

partial_tokenizer = partial(_tokenizer, tokenizer=tokenizer, window=WINDOW, max_lenght=MAX_POS_EMBD-2)

In [18]:
dataset['text'][:5]

['Ni! Ni! Sou o Solstag, também conhecido por Ale. Moro em São Paulo, onde faço pesquisa e dou aulas, contribuo com a Wikiversidade e modestamente com a Wikipédia. Participo de alguns mutirões da Wikimedia Brasil, como: Cidades Sustentáveis e suas ramificações Arquigrafia Bienal Internacional de Arte de São Paulo Fora do Eixo Museu Afro Digital Wikimedia Foundation Universidade de São Paulo e suas ramificações São Paulo Colabora Sexta poética Reverberações Wikimedia:Bugzilla Mutirões dos quais já participei: Campus Party Sesc Pompéia Também sou admin nesta wiki, caso necessite de alguma ação administrativa pode me consultar. Em geral também estou no canal de IRC; acompanho cada vez menos a lista de discussão. /Várzea Tarefas gerais arquivar de maneira clara a página Wikimedia Brasil, trazida do Meta, e as suas subpáginas Onde estamos agora?',
 'Mudanças na PP Gostei, apenas penso que os "projetos irmãos" deveriam aparecer depois dos "projetos de conteúdo". Abraços, Pietro Roveri 16h16m

In [17]:
dataset = dataset.map(partial_tokenizer, batched=True, num_proc=4)

TypeError: expected bytes, int found

In [ ]:
# data_collator = DataCollatorWithPadding(
#     tokenizer=tok, padding="max_length", max_length=MAX_LENGTH
# )

In [ ]:
# train_loader = DataLoader(DatasetKGC(train), batch_size=BATCH_SIZE, shuffle=False)